Experiment shows differences in use different costs functions on wine dataset

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import time
import os
from rf_counterfactuals import RandomForestExplainer, visualize, evaluate_counterfactual, evaluate_counterfactual_set

from collections import defaultdict


from sklearn import preprocessing
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

DATASET_PATH = "./datasets/"

In [ ]:
dataset = pd.read_csv(os.path.join(DATASET_PATH, "wine_data.csv"), sep=',')

dataset.columns = ['quality', 'alcohol', 'malic acid', 'ash', 'alcalinity of ash', 'magnesium', 'total phenols', 'flavanoids',
                  'nonflavanoid phenols', 'proanthocyanis', 'color intensity', 'hue', 'OD280/OD315 of diluted wines', 'proline'
                  ]

class_feature = "quality"

fig = plt.figure(figsize=(10, 6))
bars = plt.bar(['1', '2', '3'], dataset[class_feature].value_counts()[dataset[class_feature].unique()])
fig.gca().bar_label(bars)
plt.xlabel("Label")
plt.ylabel("Count")
plt.title("Wine dataset. Class labels distribution")
plt.plot()
plt.show()

# 30 trees

In [ ]:
K = iter([3, 5])
N_ESTIMATORS = 30

METRICS = []
METRICS += ['euclidean', 'cosine', 'jaccard', 'pearson_correlation', 'unmatched_components', 'hoem', 'implausibility_single']
METRICS += ['k_nearest_neighborhood', 'k_nearest_neighborhood']

RETRY = 1

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1000)
X = dataset.loc[:, dataset.columns!=class_feature]
y = dataset[class_feature]

cfs2_dict = {}
cfs3_dict = {}
times2 = {}
times3 = {}
times23 = {}

scores2 = []
scores3 = []
scores23 = []

accuraces30 = []

START = time.time()

for metric in METRICS:
    cfs2_dict[metric] = []
    cfs3_dict[metric] = []
    times2[metric] = []
    times3[metric] = []
    times23[metric] = []
    
    s2 = []
    s3 = []
    s23 = []
    
    k = 5
    if "nearest_neighborhood" in metric:
        k = next(K)

    print(metric, k)


    split = 0
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        print(f"split {split+1}/5")
        split += 1
        print(y_test.value_counts())
        
        X_test_label_1 = X_test[y_test==1]
        X_test_label_2 = X_test[y_test==2]

        for i in range(RETRY):
            print(f"{i+1}/{RETRY}")
            
            rf = RandomForestClassifier(n_estimators=N_ESTIMATORS, max_depth=None, random_state=i*10+1000)
            rf.fit(X_train, y_train)
            acc_score = accuracy_score(rf.predict(X_test), y_test)
            accuraces30.append(acc_score)
#             print(acc_score)

            rfe = RandomForestExplainer(rf, X_train, y_train)
            start_time = time.time()
            cfs_2 = rfe.explain_with_single_metric(X_test_label_1, 2, k=k, metric=metric, limit=1)
            end_time = time.time()

            times2[metric] += [end_time - start_time]
            print(f"Total counterfactuals found: {sum([len(c) for c in cfs_2])}")
            print(f"Finished in {end_time - start_time: 1.4f}s")
            
            for i in range(len(cfs_2)):
                if len(cfs_2[i]) == 0:
                    continue
                e = evaluate_counterfactual(rfe, X_test_label_1.iloc[i], cfs_2[i].iloc[0], 5)
                s2.append([e['sparsity'], e['proximity'], e['implausibility']])
            

            rfe = RandomForestExplainer(rf, X_train, y_train)
            start_time = time.time()
            cfs_3 = rfe.explain_with_single_metric(X_test_label_1, 3, k=k, metric=metric, limit=1)
            end_time = time.time()

            times3[metric] += [end_time - start_time]
            print(f"Total counterfactuals found: {sum([len(c) for c in cfs_3])}")
            print(f"Finished in {end_time - start_time: 1.4f}s")
            
            for i in range(len(cfs_3)):
                if len(cfs_3[i]) == 0:
                    continue
                e = evaluate_counterfactual(rfe, X_test_label_1.iloc[i], cfs_3[i].iloc[0], 5)
                s3.append([e['sparsity'], e['proximity'], e['implausibility']])
                
                
                
            
    means, stds = np.mean(np.array(s2), axis=0), np.std(np.array(s2), axis=0)
    scores2.append([means, stds])

    means, stds = np.mean(np.array(s3), axis=0), np.std(np.array(s3), axis=0)
    scores3.append([means, stds])
    
#     means, stds = np.mean(np.array(s23), axis=0), np.std(np.array(s23), axis=0)
#     scores23.append([means, stds])
    
    means, stds = np.mean(times2[metric]), np.std(times2[metric])
    times2[metric] = [means, stds]
    means, stds = np.mean(times3[metric]), np.std(times3[metric])
    times3[metric] = [means, stds]
#     means, stds = np.mean(times23[metric]), np.std(times23[metric])
#     times23[metric] = [means, stds]

    
    
END = time.time()

print(f"TOTAL TIME = {END - START}s")
    
scores2 = np.array(scores2)
scores3 = np.array(scores3)
scores23 = np.array(scores23)
scores3.shape 



In [ ]:
K = iter([3, 5])

for no, metric in enumerate(METRICS):
    m = metric.replace("_", "\\_")
    if 'nearest' in m:
        m = f"{next(K)}" + m[1:]
    print(f"{m} & {scores2[no, 0, 0]:1.3f}({scores2[no, 1, 0]:1.3f}) & {scores2[no, 0, 1]:1.3f}({scores2[no, 1, 1]:1.3f}) & {scores2[no, 0, 2]:1.3f}({scores2[no, 1, 2]:1.3f}) \\\\")
    
for no, metric in enumerate(METRICS):
    print(f"{times2[metric]}")
    
    
K = iter([3, 5])
for no, metric in enumerate(METRICS):
    m = metric.replace("_", "\\_")
    if 'nearest' in m:
        m = f"{next(K)}" + m[1:]
    print(f"{m} & {scores3[no, 0, 0]:1.3f}({scores3[no, 1, 0]:1.3f}) & {scores3[no, 0, 1]:1.3f}({scores3[no, 1, 1]:1.3f}) & {scores3[no, 0, 2]:1.3f}({scores3[no, 1, 2]:1.3f}) \\\\")
    
for no, metric in enumerate(METRICS):
    print(f"{times3[metric]}")

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 16))

width=0.3
y_pos = np.arange(len(METRICS))

for no, met in enumerate(['sparsity', 'proximity', 'implausibility']):
    label = met
    if 'implausibility' in label:
        label += '(k=5)'
    ax[0].barh(y_pos + (no-1)*width, scores2[:, 0, no], width, xerr=scores2[:, 1, no], align='center', label=label)
    ax[1].barh(y_pos + (no-1)*width, scores3[:, 0, no], width, xerr=scores3[:, 1, no], align='center', label=label)


    
for i in range(2):
    ax[i].set_xlim([0.0, 1.5])
    metrics = METRICS.copy()
    metrics[-3] = metrics[-3] + '(k=5)'
    metrics[-2] = '3' + metrics[-2][1:]
    metrics[-1] = '5' + metrics[-1][1:]
    ax[i].set_yticks(y_pos, labels=metrics)
    ax[i].legend()
    ax[i].grid()
    ax[i].invert_yaxis()  # labels read top-to-bottom
    ax[i].set_xlabel('Value')
    ax[i].set_ylabel("Metric")
    ax[i].set_title(f"Random forest (n_estimators={N_ESTIMATORS}). 5-fold-cross-validation. Label change: '1' -> '{i+2}'")


plt.show()

# 100 trees

In [ ]:
K = iter([3, 5])
N_ESTIMATORS = 100

METRICS = []
METRICS += ['euclidean', 'cosine', 'jaccard', 'pearson_correlation', 'unmatched_components', 'hoem', 'implausibility_single']
METRICS += ['k_nearest_neighborhood', 'k_nearest_neighborhood']

RETRY = 1

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1000)
X = dataset.loc[:, dataset.columns!=class_feature]
y = dataset[class_feature]

cfs2_dict = {}
cfs3_dict = {}
times2 = {}
times3 = {}
times23 = {}

scores2 = []
scores3 = []
scores23 = []

accuraces100 = []

START = time.time()

for metric in METRICS:
    cfs2_dict[metric] = []
    cfs3_dict[metric] = []
    times2[metric] = []
    times3[metric] = []
    times23[metric] = []
    
    s2 = []
    s3 = []
    s23 = []
    
    k = 5
    if "nearest_neighborhood" in metric:
        k = next(K)

    print(metric, k)


    split = 0
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        print(f"split {split+1}/5")
        split += 1
        print(y_test.value_counts())
        
        X_test_label_1 = X_test[y_test==1]
        X_test_label_2 = X_test[y_test==2]

        for i in range(RETRY):
            print(f"{i+1}/{RETRY}")
            
            rf = RandomForestClassifier(n_estimators=N_ESTIMATORS, max_depth=None, random_state=i*10+1000)
            rf.fit(X_train, y_train)
            acc_score = accuracy_score(rf.predict(X_test), y_test)
            accuraces100.append(acc_score)
#             print(acc_score)

            rfe = RandomForestExplainer(rf, X_train, y_train)
            start_time = time.time()
            cfs_2 = rfe.explain_with_single_metric(X_test_label_1, 2, k=k, metric=metric, limit=1)
            end_time = time.time()

            times2[metric] += [end_time - start_time]
            print(f"Total counterfactuals found: {sum([len(c) for c in cfs_2])}")
            print(f"Finished in {end_time - start_time: 1.4f}s")
            
            for i in range(len(cfs_2)):
                if len(cfs_2[i]) == 0:
                    continue
                e = evaluate_counterfactual(rfe, X_test_label_1.iloc[i], cfs_2[i].iloc[0], 5)
                s2.append([e['sparsity'], e['proximity'], e['implausibility']])
            

            rfe = RandomForestExplainer(rf, X_train, y_train)
            start_time = time.time()
            cfs_3 = rfe.explain_with_single_metric(X_test_label_1, 3, k=k, metric=metric, limit=1)
            end_time = time.time()

            times3[metric] += [end_time - start_time]
            print(f"Total counterfactuals found: {sum([len(c) for c in cfs_3])}")
            print(f"Finished in {end_time - start_time: 1.4f}s")
            
            for i in range(len(cfs_3)):
                if len(cfs_3[i]) == 0:
                    continue
                e = evaluate_counterfactual(rfe, X_test_label_1.iloc[i], cfs_3[i].iloc[0], 5)
                s3.append([e['sparsity'], e['proximity'], e['implausibility']])
                
                
                
            
    means, stds = np.mean(np.array(s2), axis=0), np.std(np.array(s2), axis=0)
    scores2.append([means, stds])

    means, stds = np.mean(np.array(s3), axis=0), np.std(np.array(s3), axis=0)
    scores3.append([means, stds])
    
#     means, stds = np.mean(np.array(s23), axis=0), np.std(np.array(s23), axis=0)
#     scores23.append([means, stds])
    
    means, stds = np.mean(times2[metric]), np.std(times2[metric])
    times2[metric] = [means, stds]
    means, stds = np.mean(times3[metric]), np.std(times3[metric])
    times3[metric] = [means, stds]
#     means, stds = np.mean(times23[metric]), np.std(times23[metric])
#     times23[metric] = [means, stds]

    
    
END = time.time()

print(f"TOTAL TIME = {END - START}s")
    
scores2 = np.array(scores2)
scores3 = np.array(scores3)
scores23 = np.array(scores23)
scores3.shape 



In [ ]:
K = iter([3, 5])

for no, metric in enumerate(METRICS):
    m = metric.replace("_", "\\_")
    if 'nearest' in m:
        m = f"{next(K)}" + m[1:]
    print(f"{m} & {scores2[no, 0, 0]:1.3f}({scores2[no, 1, 0]:1.3f}) & {scores2[no, 0, 1]:1.3f}({scores2[no, 1, 1]:1.3f}) & {scores2[no, 0, 2]:1.3f}({scores2[no, 1, 2]:1.3f}) \\\\")
    
for no, metric in enumerate(METRICS):
    print(f"{times2[metric]}")
    
    
K = iter([3, 5])
for no, metric in enumerate(METRICS):
    m = metric.replace("_", "\\_")
    if 'nearest' in m:
        m = f"{next(K)}" + m[1:]
    print(f"{m} & {scores3[no, 0, 0]:1.3f}({scores3[no, 1, 0]:1.3f}) & {scores3[no, 0, 1]:1.3f}({scores3[no, 1, 1]:1.3f}) & {scores3[no, 0, 2]:1.3f}({scores3[no, 1, 2]:1.3f}) \\\\")
    
for no, metric in enumerate(METRICS):
    print(f"{times3[metric]}")

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 16))

width=0.3
y_pos = np.arange(len(METRICS))

for no, met in enumerate(['sparsity', 'proximity', 'implausibility']):
    label = met
    if 'implausibility' in label:
        label += '(k=5)'
    ax[0].barh(y_pos + (no-1)*width, scores2[:, 0, no], width, xerr=scores2[:, 1, no], align='center', label=label)
    ax[1].barh(y_pos + (no-1)*width, scores3[:, 0, no], width, xerr=scores3[:, 1, no], align='center', label=label)


    
for i in range(2):
    ax[i].set_xlim([0.0, 1.5])
    metrics = METRICS.copy()
    metrics[-3] = metrics[-3] + '(k=5)'
    metrics[-2] = '3' + metrics[-2][1:]
    metrics[-1] = '5' + metrics[-1][1:]
    ax[i].set_yticks(y_pos, labels=metrics)
    ax[i].legend()
    ax[i].grid()
    ax[i].invert_yaxis()  # labels read top-to-bottom
    ax[i].set_xlabel('Value')
    ax[i].set_ylabel("Metric")
    ax[i].set_title(f"Random forest (n_estimators={N_ESTIMATORS}). 5-fold-cross-validation. Label change: '1' -> '{i+2}'")


plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))
plt.boxplot([accuraces30, accuraces])

plt.xticks([1, 2], ['30', '100'])
plt.xlabel("n_estimators")
plt.ylabel("Accuracy")
plt.title("Wine dataset. Accuracy distribition on 5-fold-cv test data")
plt.show()

